# Joint Bert Model for slot and intent classification

### Imports

In [6]:
import json
import os
import re
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, CategoricalAccuracy, BinaryAccuracy

model_name = "bert-base-uncased"

# connect MLFlow
import mlflow
mlflow.login()

# set the experiment id
mlflow.set_experiment(experiment_id="1400101170489055")

mlflow.tensorflow.autolog()

2024/10/06 13:55:29 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://community.cloud.databricks.com.


### Load Dataset

In [7]:
inputs = []
intentOutputs = []
slotOutputs = []

with open("../processing/JERTmate_final_data.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

    inputs = data["inputs"]
    intentOutputs = data["intentOutputs"]
    slotOutputs = data["slotOutputs"]

### Split Data - Train 80% | Validation 10% | Test 10%

In [8]:
def split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio):
    assert len(inputs) == len(intentOutputs) == len(slotOutputs), "All arrays must have the same length"
    
    n_total = len(inputs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    
    # split inputs
    inputs_train, inputs_val, inputs_test = inputs[:n_train], inputs[n_train:n_train + n_val], inputs[n_train + n_val:]

    # split intents
    intentOutputs_train, intentOutputs_val, intentOutputs_test = intentOutputs[:n_train], intentOutputs[n_train:n_train + n_val], intentOutputs[n_train + n_val:]

    # split slots
    slot_type_map_train, slot_type_map_val, slot_type_map_test = [x[:50] for x in slotOutputs[:n_train]], [x[:50] for x in slotOutputs[n_train:n_train + n_val]], [x[:50] for x in slotOutputs[n_train + n_val:]]
    slot_intent_map_train, slot_intent_map_val, slot_intent_map_test = [x[50:100] for x in slotOutputs[:n_train]], [x[50:100] for x in slotOutputs[n_train:n_train + n_val]], [x[50:100] for x in slotOutputs[n_train + n_val:]]
    slot_action_map_train, slot_action_map_val, slot_action_map_test = [x[100:150] for x in slotOutputs[:n_train]], [x[100:150] for x in slotOutputs[n_train:n_train + n_val]], [x[100:150] for x in slotOutputs[n_train + n_val:]]
    slot_pointers_map_train, slot_pointers_map_val, slot_pointers_map_test = [x[150:300] for x in slotOutputs[:n_train]], [x[150:300] for x in slotOutputs[n_train:n_train + n_val]], [x[150:300] for x in slotOutputs[n_train + n_val:]]
    phantom_target_map_train, phantom_target_map_val, phantom_target_map_test = [x[300:305] for x in slotOutputs[:n_train]], [x[300:305] for x in slotOutputs[n_train:n_train + n_val]], [x[300:305] for x in slotOutputs[n_train + n_val:]]
    phantom_intent_map_train, phantom_intent_map_val, phantom_intent_map_test = [x[305:310] for x in slotOutputs[:n_train]], [x[305:310] for x in slotOutputs[n_train:n_train + n_val]], [x[305:310] for x in slotOutputs[n_train + n_val:]]
    phantom_action_map_train, phantom_action_map_val, phantom_action_map_test = [x[310:315] for x in slotOutputs[:n_train]], [x[310:315] for x in slotOutputs[n_train:n_train + n_val]], [x[310:315] for x in slotOutputs[n_train + n_val:]]
    phantom_pointers_map_train, phantom_pointers_map_val, phantom_pointers_map_test = [x[315:] for x in slotOutputs[:n_train]], [x[315:] for x in slotOutputs[n_train:n_train + n_val]], [x[315:] for x in slotOutputs[n_train + n_val:]]

    
    return (tf.constant(inputs_train), tf.constant(inputs_val), tf.constant(inputs_test)), (tf.constant(intentOutputs_train), tf.constant(intentOutputs_val), tf.constant(intentOutputs_test)), (tf.constant(slot_type_map_train), tf.constant(slot_type_map_val), tf.constant(slot_type_map_test)), (tf.constant(slot_intent_map_train), tf.constant(slot_intent_map_val), tf.constant(slot_intent_map_test)), (tf.constant(slot_action_map_train), tf.constant(slot_action_map_val), tf.constant(slot_action_map_test)), (tf.constant(slot_pointers_map_train), tf.constant(slot_pointers_map_val), tf.constant(slot_pointers_map_test)), (tf.constant(phantom_target_map_train), tf.constant(phantom_target_map_val), tf.constant(phantom_target_map_test)), (tf.constant(phantom_intent_map_train), tf.constant(phantom_intent_map_val), tf.constant(phantom_intent_map_test)), (tf.constant(phantom_action_map_train), tf.constant(phantom_action_map_val), tf.constant(phantom_action_map_test)), (tf.constant(phantom_pointers_map_train), tf.constant(phantom_pointers_map_val), tf.constant(phantom_pointers_map_test))


train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

print(len(inputs))

(inputs_train, inputs_val, inputs_test), (intentOutputs_train, intentOutputs_val, intentOutputs_test), (slot_type_map_train, slot_type_map_val, slot_type_map_test), (slot_intent_map_train, slot_intent_map_val, slot_intent_map_test), (slot_action_map_train, slot_action_map_val, slot_action_map_test), (slot_pointers_map_train, slot_pointers_map_val, slot_pointers_map_test), (phantom_target_map_train, phantom_target_map_val, phantom_target_map_test), (phantom_intent_map_train, phantom_intent_map_val, phantom_intent_map_test), (phantom_action_map_train, phantom_action_map_val, phantom_action_map_test), (phantom_pointers_map_train, phantom_pointers_map_val, phantom_pointers_map_test) = split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio)

print(intentOutputs_train.shape)

58517
(35110, 38)


### Define Model

In [9]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_vector_length=None, total_slot_number=None, total_phantom_slot_number=None, slot_types=None, slot_intents=None, pointer_possibilities=None, model_name=model_name, dropout_prob=0.08):
        super().__init__(name="joint_intent_slot")
        #   ** GENERAL LAYERS **
        self.bert = TFBertModel.from_pretrained(model_name) # BERT model
        self.dropout = Dropout(dropout_prob) # basic dropout layer
        self.flatten = Flatten() # flatten layer



        #   ** SLOT LAYERS **
        # LHS compressor
        #?self.conv_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 768), strides=(1, 1),padding='valid',activation='relu')
        self.last_hidden_sequence_compressor = Dense(450, activation="relu", name="last_hidden_sequence_compressor")

        # processing layers
        #?self.slot_processor = Dense(((150) + (intent_vector_length * 2) + (slot_vector_length * 2)), activation="relu", name="slot_processor")

        # slot output layers
        self.slot_type_dense = Dense(total_slot_number * slot_types, activation='softmax', name="slot_type_output")
        self.slot_type_reshape = Reshape((total_slot_number, slot_types))
        
        self.slot_intent_dense = Dense(total_slot_number * slot_intents, activation='softmax', name="slot_intent_output")
        self.slot_intent_reshape = Reshape((total_slot_number, slot_intents))
        
        self.slot_action_output = Dense(total_slot_number, activation='sigmoid', name="slot_action_output")
        
        self.slot_pointers_dense = Dense(total_slot_number * pointer_possibilities * 3, activation='softmax', name="slot_pointers_output")
        self.slot_pointers_reshape = Reshape((total_slot_number * 3, pointer_possibilities))

        # Phantom slot output layers
        self.phantom_slot_target_dense = Dense(total_phantom_slot_number * pointer_possibilities, activation='softmax', name="phantom_slot_target_output")
        self.phantom_slot_target_reshape = Reshape((total_phantom_slot_number, pointer_possibilities))
        
        self.phantom_slot_intent_dense = Dense(total_phantom_slot_number * slot_intents, activation='softmax', name="phantom_slot_intent_output")
        self.phantom_slot_intent_reshape = Reshape((total_phantom_slot_number, slot_intents))
        
        self.phantom_slot_action_output = Dense(total_phantom_slot_number, activation='sigmoid', name="phantom_slot_action_output")
        
        self.phantom_slot_pointers_dense = Dense(total_phantom_slot_number * pointer_possibilities * 3, activation='softmax', name="phantom_slot_pointers_output")
        self.phantom_slot_pointers_reshape = Reshape((total_phantom_slot_number * 3, pointer_possibilities))



        #  ** INTENT LAYERS **
        # processing layers
        #?self.intent_processor = Dense(((150 * 1) + (intent_vector_length * 2)), activation="relu", name="intent_processor")

        # output layer
        self.intent_output = Dense(intent_vector_length, activation='softmax', name="intent_output")

    def __call__(self, inputs, **kwargs):
        # run BERT
        trained_bert = self.bert(inputs[:, :150], **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state

        #   ** SLOT CLASSIFICATION **
        # reduce dimensionality of sequence output - Dense(150)
        flattened_sequence_output = self.flatten(sequence_output)
        LHSC_output = self.last_hidden_sequence_compressor(flattened_sequence_output)

        # slot processor
        #slot_processor_input = self.dropout(tf.concat([LHSC_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        #slot_processor_output = self.slot_processor(slot_processor_input)

        # slot output
        slot_output_input = self.dropout(tf.concat([LHSC_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        
        slot_type_output = self.slot_type_dense(slot_output_input)
        slot_type_output = self.slot_type_reshape(slot_type_output)
        
        slot_intent_output = self.slot_intent_dense(slot_output_input)
        slot_intent_output = self.slot_intent_reshape(slot_intent_output)
        
        slot_action_output = self.slot_action_output(slot_output_input)
        
        slot_pointers_output = self.slot_pointers_dense(slot_output_input)
        slot_pointers_output = self.slot_pointers_reshape(slot_pointers_output)

        # Phantom slot outputs
        phantom_target_output = self.phantom_slot_target_dense(slot_output_input)
        phantom_target_output = self.phantom_slot_target_reshape(phantom_target_output)
        
        phantom_intent_output = self.phantom_slot_intent_dense(slot_output_input)
        phantom_intent_output = self.phantom_slot_intent_reshape(phantom_intent_output)
        
        phantom_action_output = self.phantom_slot_action_output(slot_output_input)
        
        phantom_pointers_output = self.phantom_slot_pointers_dense(slot_output_input)
        phantom_pointers_output = self.phantom_slot_pointers_reshape(phantom_pointers_output)



        #   ** INTENT CLASSIFICATION **
        # intent processor
        #intent_processor_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        #intent_processor_output = self.intent_processor(intent_processor_input)

        # intent output
        intent_output_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        intent_output = self.intent_output(intent_output_input)

        return intent_output, slot_type_output, slot_intent_output, slot_action_output, slot_pointers_output, phantom_target_output, phantom_intent_output, phantom_action_output, phantom_pointers_output

    def get_config(self):
        config = super(JointIntentAndSlotFillingModel, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
joint_model = JointIntentAndSlotFillingModel(intent_vector_length=38, total_slot_number=50, total_phantom_slot_number=5, slot_types=15, slot_intents=4, pointer_possibilities=18)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Train and Compile

In [10]:
# optimizer
opt = Adam(learning_rate=6e-5, epsilon=1e-08)

# learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * 0.97
learning_rate_callback = LearningRateScheduler(scheduler)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="model_epoch_{epoch:02d}.keras",  # Save the model with the epoch number in the filename
    save_freq='epoch',
    verbose=1 
)

# loss functions
losses = [
    CategoricalCrossentropy(name="intent_loss"), 
    SparseCategoricalCrossentropy(from_logits=True, name="slot_type_loss"),
    SparseCategoricalCrossentropy(from_logits=True, name="slot_intent_loss"),
    BinaryCrossentropy(name="slot_actionable_loss"), 
    SparseCategoricalCrossentropy(from_logits=True, name="slot_pointer_loss"),
    SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_target_loss"),
    SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_intent_loss"),
    BinaryCrossentropy(name="phantom_slot_actionable_loss"), 
    SparseCategoricalCrossentropy(from_logits=True, name="phantom_slot_pointer_loss"),
]
# metrics
metrics = [
    CategoricalAccuracy(name="intent_accuracy"), 
    SparseCategoricalAccuracy(name="slot_type_accuracy"),
    SparseCategoricalAccuracy(name="slot_intent_accuracy"),
    BinaryAccuracy(name="slot_actionable_accuracy"), 
    SparseCategoricalAccuracy(name="slot_pointer_accuracy"),
    SparseCategoricalAccuracy(name="phantom_slot_target_accuracy"),
    SparseCategoricalAccuracy(name="phantom_slot_intent_accuracy"),
    BinaryAccuracy(name="phantom_slot_actionable_accuracy"), 
    SparseCategoricalAccuracy(name="phantom_slot_pointer_accuracy"),
]

# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

history = joint_model.fit(
    x=inputs_train, 
    y=(intentOutputs_train, slot_type_map_train, slot_intent_map_train, slot_action_map_train, slot_pointers_map_train, phantom_target_map_train, phantom_intent_map_train, phantom_action_map_train, phantom_pointers_map_train), 
    validation_data=(inputs_val, (intentOutputs_val, slot_type_map_val, slot_intent_map_val, slot_action_map_val, slot_pointers_map_val, phantom_target_map_val, phantom_intent_map_val, phantom_action_map_val, phantom_pointers_map_val)), 
    epochs=24, 
    batch_size=30,
    shuffle=True, 
    callbacks=[learning_rate_callback, checkpoint_callback],
)

2024/10/06 13:55:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a6bb372cc80342deac91a29ca05c6bdd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow
2024/10/06 13:55:39 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: 'JointIntentAndSlotFillingModel' object has no attribute 'input_shape'
2024/10/06 13:55:39 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: If 'features' is a TensorFlow Tensor, then 'targets' must also be a TensorFlow Tensor. Found: <class 'tuple'>.


Epoch 1/24
1171/1171 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - intent_accuracy: 0.2952 - loss: 16.5248 - phantom_slot_actionable_accuracy: 0.9970 - phantom_slot_intent_accuracy: 0.7676 - phantom_slot_pointer_accuracy: 0.2276 - phantom_slot_target_accuracy: 0.2429 - slot_actionable_accuracy: 0.9712 - slot_intent_accuracy: 0.2421 - slot_pointer_accuracy: 0.3357 - slot_type_accuracy: 0.2145
Epoch 1: saving model to model_epoch_01.keras


c:\Users\falkt\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_api.py:102: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)
Uploading C:\Users\falkt\AppData\Local\Temp\tmptiydcmfz\latest_checkpoint.h5:  85%|████████▍ | 550M/649M [00:09<00:01, 66.4MiB/s] WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmptiydcmfz\latest_checkpoint.h5:  86%|████████▋ | 560M/649M [00:09<00:01, 66.4MiB/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmptiydcmfz\latest_checkpoint.h5:  88%|████████▊ 

1171/1171 ━━━━━━━━━━━━━━━━━━━━ 3697s 3s/step - intent_accuracy: 0.2953 - loss: 16.5245 - phantom_slot_actionable_accuracy: 0.9970 - phantom_slot_intent_accuracy: 0.7676 - phantom_slot_pointer_accuracy: 0.2276 - phantom_slot_target_accuracy: 0.2430 - slot_actionable_accuracy: 0.9712 - slot_intent_accuracy: 0.2421 - slot_pointer_accuracy: 0.3357 - slot_type_accuracy: 0.2145 - val_intent_accuracy: 0.4482 - val_loss: 15.9259 - val_phantom_slot_actionable_accuracy: 1.0000 - val_phantom_slot_intent_accuracy: 0.7979 - val_phantom_slot_pointer_accuracy: 0.2211 - val_phantom_slot_target_accuracy: 0.3967 - val_slot_actionable_accuracy: 0.9819 - val_slot_intent_accuracy: 0.2742 - val_slot_pointer_accuracy: 0.4867 - val_slot_type_accuracy: 0.2421 - learning_rate: 6.0000e-05
Epoch 2/24
1171/1171 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - intent_accuracy: 0.4510 - loss: 15.9455 - phantom_slot_actionable_accuracy: 1.0000 - phantom_slot_intent_accuracy: 0.7743 - phantom_slot_pointer_accuracy: 0.2843 - phantom_

Uploading C:\Users\falkt\AppData\Local\Temp\tmpjxgdzo9z\latest_checkpoint.h5:  85%|████████▍ | 550M/649M [00:09<00:01, 78.9MiB/s] WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmpjxgdzo9z\latest_checkpoint.h5:  86%|████████▋ | 560M/649M [00:09<00:01, 78.9MiB/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmpjxgdzo9z\latest_checkpoint.h5:  88%|████████▊ | 570M/649M [00:09<00:01, 78.9MiB/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmpjxgdzo9z\latest_checkpoint.h5:  89%|████████▉ | 580M/649M [00:09<00:00, 107M

...

Uploading C:\Users\falkt\AppData\Local\Temp\tmpx5_pqst7\latest_checkpoint.h5:  85%|████████▍ | 550M/649M [00:09<00:01, 92.0MiB/s] WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmpx5_pqst7\latest_checkpoint.h5:  86%|████████▋ | 560M/649M [00:09<00:01, 92.0MiB/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmpx5_pqst7\latest_checkpoint.h5:  88%|████████▊ | 570M/649M [00:09<00:00, 92.0MiB/s]WARNING:urllib3.connectionpool:Connection pool is full, discarding connection: databricks-prod-storage-oregon.s3.us-west-2.amazonaws.com. Connection pool size: 10
Uploading C:\Users\falkt\AppData\Local\Temp\tmpx5_pqst7\latest_checkpoint.h5:  89%|████████▉ | 580M/649M [00:09<00:00, 106M

1171/1171 ━━━━━━━━━━━━━━━━━━━━ 3771s 3s/step - intent_accuracy: 0.5814 - loss: 15.0976 - phantom_slot_actionable_accuracy: 1.0000 - phantom_slot_intent_accuracy: 1.0000 - phantom_slot_pointer_accuracy: 0.9971 - phantom_slot_target_accuracy: 1.0000 - slot_actionable_accuracy: 0.9805 - slot_intent_accuracy: 0.9669 - slot_pointer_accuracy: 0.9519 - slot_type_accuracy: 0.8715 - val_intent_accuracy: 0.5890 - val_loss: 15.4759 - val_phantom_slot_actionable_accuracy: 1.0000 - val_phantom_slot_intent_accuracy: 1.0000 - val_phantom_slot_pointer_accuracy: 0.9984 - val_phantom_slot_target_accuracy: 1.0000 - val_slot_actionable_accuracy: 0.9793 - val_slot_intent_accuracy: 0.9689 - val_slot_pointer_accuracy: 0.9577 - val_slot_type_accuracy: 0.8676 - learning_rate: 4.8479e-05
Epoch 23/24
1171/1171 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - intent_accuracy: 0.5831 - loss: 15.3777 - phantom_slot_actionable_accuracy: 1.0000 - phantom_slot_intent_accuracy: 1.0000 - phantom_slot_pointer_accuracy: 0.9985 - phantom

2024/10/07 14:42:44 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id a6bb372cc80342deac91a29ca05c6bdd: Failed to log run data: Exception: TEMPORARILY_UNAVAILABLE: The service at /api/2.0/mlflow/runs/log-batch is temporarily unavailable. Please try again later. [TraceId: -]
2024/10/07 14:42:48 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id a6bb372cc80342deac91a29ca05c6bdd: Failed to log run data: Exception: TEMPORARILY_UNAVAILABLE: The service at /api/2.0/mlflow/runs/log-batch is temporarily unavailable. Please try again later. [TraceId: -]
2024/10/07 14:42:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: TEMPORARILY_UNAVAILABLE: The service at /api/2.0/mlflow/runs/set-tag is temporarily unavailable. Please try again later. [TraceId: -]


1171/1171 ━━━━━━━━━━━━━━━━━━━━ 3767s 3s/step - intent_accuracy: 0.5812 - loss: 15.3194 - phantom_slot_actionable_accuracy: 1.0000 - phantom_slot_intent_accuracy: 1.0000 - phantom_slot_pointer_accuracy: 0.9991 - phantom_slot_target_accuracy: 1.0000 - slot_actionable_accuracy: 0.9833 - slot_intent_accuracy: 0.9772 - slot_pointer_accuracy: 0.9743 - slot_type_accuracy: 0.8977 - val_intent_accuracy: 0.5889 - val_loss: 15.4673 - val_phantom_slot_actionable_accuracy: 1.0000 - val_phantom_slot_intent_accuracy: 1.0000 - val_phantom_slot_pointer_accuracy: 0.9996 - val_phantom_slot_target_accuracy: 1.0000 - val_slot_actionable_accuracy: 0.9785 - val_slot_intent_accuracy: 0.9813 - val_slot_pointer_accuracy: 0.9800 - val_slot_type_accuracy: 0.9013 - learning_rate: 4.5614e-05


2024/10/07 14:42:48 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: tuple index out of range
2024/10/07 14:42:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/07 14:43:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\falkt\AppData\Local\Temp\tmpgk2vxwde\model, flavor: tensorflow). Fall back to return ['tensorflow==2.17.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2024/10/07 14:43:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Uploading artifacts:   0%|          | 0/7 [00:00<?, ?it/s]
2

### Evaluate Model

keep in mind -> test slot accuracy will be higher than reality.
        Because 90% of the data points are 0, it can just guess 0 and be right 85% of the time

In [11]:
test_loss, test_intent_acc, test_slot_type, test_slot_intent, test_slot_action, test_slot_pointers, test_phantom_target, test_phantom_intent, test_phantom_action, test_phantom_pointers = joint_model.evaluate(x=inputs_test, y=(intentOutputs_test, slot_type_map_test, slot_intent_map_test, slot_action_map_test, slot_pointers_map_test, phantom_target_map_test, phantom_intent_map_test, phantom_action_map_test, phantom_pointers_map_test), batch_size=30)

print(f"Test Intent Accuracy: {test_intent_acc}")
print(f"Test Slot Type Accuracy: {test_slot_type}")
print(f"Test Slot Intent Accuracy: {test_slot_intent}")
print(f"Test Slot Action Accuracy: {test_slot_action}")
print(f"Test Slot Pointers Accuracy: {test_slot_pointers}")
print(f"Test Phantom Target Accuracy: {test_phantom_target}")
print(f"Test Phantom Intent Accuracy: {test_phantom_intent}")
print(f"Test Phantom Action Accuracy: {test_phantom_action}")
print(f"Test Phantom Pointers Accuracy: {test_phantom_pointers}")

391/391 ━━━━━━━━━━━━━━━━━━━━ 830s 2s/step - intent_accuracy: 0.5873 - loss: 15.4344 - phantom_slot_actionable_accuracy: 1.0000 - phantom_slot_intent_accuracy: 1.0000 - phantom_slot_pointer_accuracy: 0.9997 - phantom_slot_target_accuracy: 1.0000 - slot_actionable_accuracy: 0.9788 - slot_intent_accuracy: 0.9832 - slot_pointer_accuracy: 0.9816 - slot_type_accuracy: 0.9034
Test Intent Accuracy: 0.5856971740722656
Test Slot Type Accuracy: 0.9026625752449036
Test Slot Intent Accuracy: 0.9825135469436646
Test Slot Action Accuracy: 0.9782508015632629
Test Slot Pointers Accuracy: 0.981245219707489
Test Phantom Target Accuracy: 1.0
Test Phantom Intent Accuracy: 1.0
Test Phantom Action Accuracy: 1.0
Test Phantom Pointers Accuracy: 0.9996238350868225


### Inference